In [79]:
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

plt.style.use('ggplot')
tqdm.pandas()

from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.models import Model
from keras.layers import *
from keras.callbacks import EarlyStopping


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Чтение и предобработка

In [80]:
with open('data/sentiment_texts.pickle', 'rb') as f1:
    sentiment_texts = pickle.load(f1)
sentiment_texts

,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward
0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,False
1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False
2,10090,1063908560,118,4,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
3,10090,1063908560,220,5,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
4,9826,1063908560,89,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False
...,...,...,...,...,...,...,...,...
9284,47482,1197210433,157,4,2023-03-20 14:53:14,2023-03-20 12:15:21,#FLOT #Дивиденды 💰 7% — возможная дивдоходност...,False
9285,233829,1203560567,157,4,2023-03-20 14:58:04,2023-03-20 12:05:49,🇷🇺#FLOT #отчетность ЧИСТАЯ ПРИБЫЛЬ СОВКОМФЛОТ...,False
9286,9789,1063908560,225,3,2023-04-19 17:51:56,2023-04-19 15:32:00,​​Ключевой принцип создания портфеля 🔹Диверси...,False
9287,233867,1203560567,127,3,2023-03-20 14:58:04,2023-03-20 14:33:32,"""💥🇷🇺#PLZL #листинг #торги """"Полюс"""" ведет диа...",False


In [81]:
sentiment_texts = sentiment_texts[sentiment_texts['SentimentScore'] != 0]
sentiment_texts

,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward
0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,False
1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False
2,10090,1063908560,118,4,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
3,10090,1063908560,220,5,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
4,9826,1063908560,89,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False
...,...,...,...,...,...,...,...,...
9284,47482,1197210433,157,4,2023-03-20 14:53:14,2023-03-20 12:15:21,#FLOT #Дивиденды 💰 7% — возможная дивдоходност...,False
9285,233829,1203560567,157,4,2023-03-20 14:58:04,2023-03-20 12:05:49,🇷🇺#FLOT #отчетность ЧИСТАЯ ПРИБЫЛЬ СОВКОМФЛОТ...,False
9286,9789,1063908560,225,3,2023-04-19 17:51:56,2023-04-19 15:32:00,​​Ключевой принцип создания портфеля 🔹Диверси...,False
9287,233867,1203560567,127,3,2023-03-20 14:58:04,2023-03-20 14:33:32,"""💥🇷🇺#PLZL #листинг #торги """"Полюс"""" ведет диа...",False


In [82]:
X = sentiment_texts['MessageText']
y = sentiment_texts['SentimentScore']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(np.unique(y_train))

[1 2 3 4 5]


In [83]:
max_length = 5000
max_features = 20000
embedding_dim = 300

x_all = []
x_all.extend(x_test)
x_all.extend(x_train)

tk = Tokenizer(num_words=max_features, lower=True, filters='\n\t')
tk.fit_on_texts(x_all)
x_train_seq = tk.texts_to_sequences(x_train)
x_test_seq = tk.texts_to_sequences(x_test)

np_x_train = pad_sequences(x_train_seq, maxlen=max_length,  padding='post')
np_x_test = pad_sequences(x_test_seq, maxlen=max_length,  padding='post')
np_y_train = to_categorical(y_train)

class_num = np_y_train.shape[1]

print(f'np_x_train shape: {np_x_train.shape}')
print(f'np_x_test shape: {np_x_test.shape}')
print(f'np_y_train shape: {np_y_train.shape}')


np_x_train shape: (7300, 5000)
np_x_test shape: (1826, 5000)
np_y_train shape: (7300, 6)


In [84]:
np_y_train[:10]

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)

# CNN-BiLSTM

In [85]:
def one_input_classifier(max_length, max_features, embedding_dim, class_num):
    inputs = Input(shape=(max_length,), name='input_1')
    embeddings = Embedding(max_features, embedding_dim, input_length=max_length, name='embedding_1')(inputs)

    conv_1 = Conv1D(32, 9, activation='relu', name='conv1d_1')(embeddings)
    maxpool_1 = MaxPooling1D(16, name='maxpool1d_1')(conv_1)
    dropout_1 = Dropout(0.2, name='dropout_1')(maxpool_1)

    conv_2 = Conv1D(32, 7, activation='relu', name='conv1d_2')(dropout_1)
    maxpool_2 = MaxPooling1D(8, name='maxpool1d_2')(conv_2)
    dropout_2 = Dropout(0.2, name='dropout_2')(maxpool_2)

    bilstm = Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.2, name='lstm_1'),
        name='bidirectional_1')(dropout_2)
    preds = Dense(class_num, activation='softmax', name='preds')(bilstm)

    model = Model(inputs=inputs, outputs=preds)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=1,
                               mode='min', baseline=None, restore_best_weights=True)

models = []
classifier_num = 1

for i in range(classifier_num):
    model = one_input_classifier(max_length, max_features, embedding_dim, class_num)

    if i == 0:
        print(model.summary())

    model.fit(np_x_train, np_y_train, validation_split=0.3, shuffle=True,
              callbacks=[early_stopping], epochs=10, batch_size=32, verbose=1)
    models.append(model)

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5000)]            0         
                                                                 
 embedding_1 (Embedding)     (None, 5000, 300)         6000000   
                                                                 
 conv1d_1 (Conv1D)           (None, 4992, 32)          86432     
                                                                 
 maxpool1d_1 (MaxPooling1D)  (None, 312, 32)           0         
                                                                 
 dropout_1 (Dropout)         (None, 312, 32)           0         
                                                                 
 conv1d_2 (Conv1D)           (None, 306, 32)           7200      
                                                                 
 maxpool1d_2 (MaxPooling1D)  (None, 38, 32)            0   

# Evaluation

In [86]:
y_pred = models[0].predict(np_x_test, batch_size=32, verbose=1)

58/58 [==============================] - 43s 717ms/step


In [87]:
def final_score(y_test, y_pred):
    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)
    return (f1 + accuracy) / 2

In [88]:
predicted_labels = np.argmax(y_pred, axis=1)
predicted_labels.shape

(1826,)

In [89]:
print(final_score(y_test, predicted_labels))

0.5916779122127136


In [90]:
models[0].save('my_model1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
